# Making money with portfolio optimization
One of the great applications of mathematical optimization is portfolio optimization, which was pioneered in 1952 by [Harry Markowitz](https://en.wikipedia.org/wiki/Harry_Markowitz) in his seminal [paper](https://www.math.ust.hk/~maykwok/courses/ma362/07F/markowitz_JF.pdf) that earned him a Nobel prize in 1990. The essence of it is that when making investment decisions, you should not simply go by the highest mean return on investment, but you also have to consider the risk associated with this.

Specifically, we are interested in making as much money while ensure with $1-\beta$ percent probability that we are not going to loose money. To model this type of  probabilistic constraint, we can use:
\begin{align*}
\bar{p}^Tx + \Phi^{-1}(\beta)\left|\left|\sum \limits_{i} \Sigma x\right|\right|_2 \geq 0,
\end{align*}
where $\Phi$ is the cumulative density function (CDF) of a unit Gaussian random variable, $\Sigma$ is the covariance matrix and $x$ is the fraction of the investment amount allocated to each investment. Note that:
\begin{equation}
\left| \left| c \right| \right|_2 = \left| \left| \begin{bmatrix} c_1 \\c_2 \\ \vdots \\ c_n \end{bmatrix} \right| \right|_2 = \sqrt{\sum \limits_i c_i^2}
\end{equation}
is called the 2-norm (or Euclidean norm).

The idea behind this constraint is the following:
- $\bar{p}^Tx$ denotes how much money we make on average. We now have to adjust for the uncertainty associated with our choice of $x$.
- $\left|\left|\sum \limits_{i} \Sigma x\right|\right|_2$ sums up the covariance weighted by our choice of investment $x$. In other words, the term represents teh *cumulative* risk we incur for a selection $x$.
- $\Phi^{-1}$ is the [quantile function](https://en.wikipedia.org/wiki/Quantile_function) and denotes "the value of the random variable such that the probability of the variable being less than or equal to that value equals the given probability". In other words, it answers the question "how much would I have to shift my mean, so that I make sure that only a fraction of $\beta$ is negative" (check out [this plot](https://en.wikipedia.org/wiki/Quantile_function#/media/File:Probit_plot.png), it may give you some clarity).

Therefore, the idea is that we force the problem to find a solution where we can make sure that we will lose money only in a $\beta$ fraction of the cases.

## Data acquisition
So let's get some data. But let's get some real data just to make it fun (I took part of this script from [here](https://www.pythonforfinance.net/2017/01/21/investment-portfolio-optimisation-with-python/)):

In [1]:
import pandas_datareader.data as web
import numpy as np
import pandas as pd
import xpress as xp
%env XPRESS=..
from scipy.stats import norm
import matplotlib as plt
from dataclasses import dataclass

stocks = ['AAPL', 'AMZN', 'MSFT', 'GOOG', 'FB', 'JNJ', 'BABA', 'JPM', 'XOM', 'WMT', 'BAC', 'PFE', 'VZ', 'PG', 'T', 'BA', 
          'CSCO', 'HD', 'INTC', 'KO', 'ORCL', 'DIS', 'C', 'MCD', 'NFLX', 'IBM', 'GE', 'TSLA', 'F', 'AMD']
 
# Download daily price data for each of the stocks in the portfolio
data = web.DataReader(stocks,data_source='yahoo',start='01/01/2010')['Adj Close']
data.sort_index(inplace=True)
 
# Convert daily stock prices into daily returns
returns = data.pct_change()
 
# Calculate mean daily return and covariance of daily returns
means = returns.mean()
cov_matrix = returns.cov()
sigma = pd.Series(np.diag(cov_matrix), index=cov_matrix.index)

# If this does not work for some reason, you can also uncomment these lines and load in a dataset I downloaded:
#df = pd.read_csv('PortfolioData.csv')
#df.set_index('Symbols', inplace=True)
#means = df['Mean']
#sigma = df['Sigma']

# Probability of loosing money - has to be < 0.5 otherwise the problem is non-convex
beta = 0.05

# Define the investment class
@dataclass(frozen=True)
class Investment:
    name: str
    mean: float
    sigma: float

investments = list()
for s in stocks:
    investments.append(Investment(s, means[s], sigma[s]))

env: XPRESS=..


At this point we assume though that all the stocks are independent. This is a limiting assumption, and the formula above works for general covariance matrices. However, to keep things simple, we stick with a diagonal covariance matrix for now.

As we learned by now, let's start with the model creation, variable definition and objective function.

## Model creation and variable definition
**What is needed here?**
> Consider that you want to assign a part of your investment to a given `Investment` (listed in `investments`)

## Objective function
**What is needed here?**
> Overall, we want to make as much money as possible on average for a given `Investment`.

## Enforcing investment amount
We a fixed amount of money we can allott to all `investments`. To make our lives easy, we have normalized it to 1, and the sum of all the investments has to add up to that: **How do you write this in code?**

Ok, so far so good. The problem is that if we simply solve this problem now, we will only go after the highest expected return and not take risk into account. To do so, we need to look at the constraint stated above:

## Enforce risk limit
Let us focus on the unknown part: $\Phi^{-1}(\beta)\left|\left|\sum \Sigma x\right|\right|$. First off, $\Phi^{-1}(\beta)$ is called the [...] and available through `norm.ppf` in Python. As for the 2-norm: for now we consider the covariance matrix $\Sigma$ to be independent (i.e. no cross-terms), and so the norm equation simplifies to $\left|\left|\sum \Sigma x\right|\right| = \sqrt{\sum \limits_i (\sigma_ix_i)^2}$. Therefore, once we arrive at this point, we would expect to write the following:

In [159]:
#enforce_risk_limit = xp.constraint(xp.Sum(inv.mean*x[inv] for inv in investments) 
# + norm.ppf(beta)*xp.Sum((inv.sigma*x[inv])**2 for inv in investments)**(1/2) >= 0)

However, if you try to solve the problem with this constraint, it throws a nasty error saying that you can't do square-root of variables. But then how should we do our SOCP constraint? Shall we simply square it?

\begin{equation}
\bar{p}^Tx+ \Phi^{-1}(\beta)\sqrt{\sum \limits_{i} (\sigma_i x_i)^2} \geq 0 \\
(\Phi^{-1}(\beta))^2(\sum \limits_{i} (\sigma_i x_i)^2) \leq (\bar{p}^Tx)^2
\end{equation}

However, it you write that into Xpress, you will get that the model is non-convex! So what?

Well, the answer is that we always have to reformulate any SOCP into a very specific cone, namely:
\begin{equation}
x \succeq 0 \leftrightarrow x_1 \geq \sqrt{\left(x_2^2 + x_3^2 + ... x_n^2\right)} = \sqrt{\sum \limits_{i=2}^{n} x_i^2}
\end{equation}

Therefore, let's create an auxiliary variable that mimics the remaining terms. Let $x_{all} = \bar{p}^Tx$. Then:
\begin{equation}
\bar{p}^Tx+ \Phi^{-1}(\beta)\left|\left| \Sigma x\right|\right|_2 \geq 0
\end{equation}

Since $\Phi^{-1}(\beta) < 0$ (due to $\beta < 0.5$), we can move it to the other side and square each side without affecting the validity of the inequality:
\begin{equation}
(\Phi^{-1}(\beta))^2(\sigma_0x_0^2 + \sigma_1x_1^2 + \sigma_2x_2^2) \leq x_{all}^2
\end{equation}

**Write this in code**

In [160]:
norm.ppf(beta)

## Solve the model and post-process the result

In [161]:
model.solve()

print(f'Solution string: {model.getProbStatusString()}')

Solution string: lp_optimal


Now that we have the solution, we should validate that we described the probability constraint correctly. How do we do this? Well, the simple answer is: we sample for a Gaussian distribution with the specified mean and standard deviation, and evaluate the constraint. It should be violated less than $\beta$ percent of the time.

In [162]:
n = 1000
samples = {inv : np.random.normal(inv.mean, inv.sigma, n) for inv in investments}
satisfied = 0
for k in range(n):
    if np.sum([samples[inv][k]*model.getSolution(x[inv]) for inv in investments]) >= 0:
        satisfied += 1

print(f'Satisfaction: {satisfied}/{n}')

Satisfaction: 945/1000


In [15]:
means.name = "Mean"
sigma.name = "Sigma"

df = pd.concat([means, sigma], axis=1)
df.to_csv('PortfolioData.csv')